# Pregunta 1 - Tarea 1
## Diego Emilio Bustamante Henríquez

### Utilidades

- `e_otp`: Encripta un texto, utilizando una llave y en base al orden de un alfabeto, mediante el protocolo OTP.

- `d_otp`: Desencripta un texto, utilizando una llave y en base al orden de un alfabeto, mediante el protocolo OTP.

- `decript`: Desencripta un texto, utilizando una llave y en base al orden de un alfabeto, mediante el protocolo OTP. Este algoritmo es más rápido ya que utiliza un diccionario y no busca el índice dentro de una lista.

- `binary_search`: Dado un array (de dos dimensiones) ordenado y el valor de un elemento, retorna el índice en el cual debería insertarse el elemento.

- `binary_insertion`: Dado un array ordenado, un elemento y su valor, utiliza binary search para insertarlo. Luego, elimina el último elemento para mantaner constante el largo del arreglo. Esta estructura simula un max-heap para k elementos. Si bien puede ser mejorada, no hace falta dado que su tamaño es del orden de las decenas. 

In [78]:
def e_otp(text, key, frequencies):
  alph = sorted(frequencies.keys())
  num_text = [alph.index(letter) for letter in text]
  num_key = [alph.index(letter) for letter in key]
  long_num_key = num_key*(len(num_text)//len(num_key)) + num_key[:len(num_text)%len(num_key)]
  return "".join([alph[(m+k)%len(alph)] for m, k in zip(num_text, long_num_key)])

def d_otp(text, key, frequencies):
  alph = sorted(frequencies.keys())
  num_text = [alph.index(letter) for letter in text]
  num_key = [alph.index(letter) for letter in key]
  long_num_key = num_key*(len(num_text)//len(num_key)) + num_key[:len(num_text)%len(num_key)]
  return "".join([alph[(m-k+len(alph))%len(alph)] for m, k in zip(num_text, long_num_key)])

def decript(text, key, alph, alph_dict):
  long_key = key*(len(text)//len(key)) + key[:len(text)%len(key)]
  return "".join([alph[(alph_dict[m]-alph_dict[k])%len(alph)] for m, k in zip(text, long_key)])

def binary_search(arr, item):
  low, high = 0, len(arr) - 1
  while (low <= high):
    mid = low + (high - low) // 2
    if (item == arr[mid][1]): return mid + 1
    elif (item > arr[mid][1]): low = mid + 1
    else: high = mid - 1
  return low

def binary_insertion(arr, elem, val):
  if val >= arr[-1][1]: return arr
  index = binary_search(arr, val)
  return arr[:index] + [(elem, val)] + arr[index:-1]

### Llaves

Ambas funciones calculan las mejores llaves para el texto cifrado, dada unas frecuencias (alfabeto) y función de distancia. Utilizan la estructura descrita para retornar los 20 mejores resultados, a veces la función entrega un resultado erróneo como mejor llave pero la real está entre estos resultados (de ahí su gran utilidad).

Las 2 funciones tienen una complejidad y estructuras similares, sin embargo, la primera tiene una mayor precisión al calcular qué tan buena es una llave.

- `statistics_best_keys`: Sigue la lógica del código visto en clases para romper OTP. Recorre todos los largos de llave. Por cada letra de una llave, elige los carácteres que desencripta y evalúa qué tan parecido a un idioma sería el texto original. Al tener todas las letras de una llave esta es guardada como candidata, su desempeño al guardarla es sobre todo el texto original.

- `greedy_best_key`: Toma una muestra del texto encriptado para reducir el costo del algoritmo. Recorre todos los largos de llave. Por cada letra de una llave, evalúa qué tan parecido a un idioma sería la muestra original. Una vez que ha probado con todas las letras, se queda con la mejor y pasa a las siguientes. Por ejemplo, se prueba AAA, BAA y CAA; si BAA es la mejor llave se probará ahora con BAA, BBA y BCA. Dado que se prueban con llaves enteras, el algoritmo es un poco más costoso por lo que el valor de una llave como candidata es su desempeño en la muestra.
 

In [79]:
def statistics_best_keys(ciphertext, frequencies, distance, alph, alph_dict):
  best_keys = [(0, 2<<30)]*20
  max_key_length = len(ciphertext)//50
  for length in range(1, max_key_length + 1):
    best_letters = [0] * length
    for index in range(length):
      sub_ciphertext, best_letter, best_dist = [ciphertext[char_index * length + index] for char_index in range(len(ciphertext) // length)], "", len(alph)
      for letter in alph:
        decripted_text = ""
        for char in sub_ciphertext: decripted_text += alph[(alph_dict[char]-alph_dict[letter]) % len(alph)]
        dist = distance(decripted_text, frequencies)
        if dist < best_dist: best_dist, best_letter = dist, letter
      best_letters[index] = best_letter
    key = "".join(best_letters)
    text = decript(ciphertext, key, alph, alph_dict)
    best_keys = binary_insertion(best_keys, key, distance(text, frequencies))
  return best_keys

def greedy_best_keys(ciphertext, frequencies, distance, alph, alph_dict):
  best_keys = [(0, 2<<30)]*20
  short_ciphertext = ciphertext[:823]
  max_key_length = min(len(ciphertext)//50, 45)
  for length in range(1, max_key_length + 1):
    best_letter, best_key_prefix, best_key_dist = "", "", 2<<30
    for index in range(length):
      best_key_prefix += best_letter
      for letter in alph:
        key = best_key_prefix + letter + alph[0]*(length - 1 - index)
        text = decript(short_ciphertext, key, alph, alph_dict)
        dist = distance(text, frequencies)
        if dist < best_key_dist: best_key_dist, best_letter = dist, letter
        best_keys = binary_insertion(best_keys, key, dist)
  return best_keys

### Break RP

- `break_rp`: utiliza una función que calcula las mejores llaves y después evalúa la mejor en el texto cifrado original. En este caso, como la función utilizada evalúa en el texto original se sabe que la primera será la mejor. En el otro caso, se deben recorrer las llaves y desencriptar el texto original (algo que toma muy poco dado que son 20 llaves). En general se encuentra la llave, si no es muy probable que la llave real esté entre las candidatas a llave. Para un texto de 7k carácteres sobre un alfabeto de 27 símbolos, tarda menos de 10 segundos en encontrar la respuesta correcta.

In [80]:
def break_rp(ciphertext, frequencies, distance):
  alph = sorted(frequencies.keys())
  alph_dict = {alph[i]:i for i in range(len(alph))}
  best_keys = statistics_best_keys(ciphertext, frequencies, distance, alph, alph_dict)
  return best_keys[0][0]

## Para testear

Utilidades para testear

In [ ]:
""""
frequencies = {
    'A': 0.082, 'B': 0.015, 'C': 0.027, 'D': 0.047, 'E': 0.13, 'F': 0.022, 'G': 0.02,
    'H': 0.062, 'I': 0.069, 'J': 0.0016, 'K': 0.0081, 'L': 0.04, 'M': 0.027, 'N': 0.067,
    'O': 0.078, 'P': 0.019, 'Q': 0.0011, 'R': 0.059, 'S': 0.062, 'T': 0.096,
    'U': 0.027, 'V': 0.0097, 'W': 0.024, 'X': 0.0015, 'Y': 0.02, 'Z': 0.00078
}
def abs_distance(string, frequencies): return sum([abs(frequencies[c] - string.count(c) / len(string)) for c in frequencies])
def clean(t): return t.replace(" ", "").replace(",", "").replace("(", "").replace(")", "").replace(";", "").replace("'", "").replace(".", "").upper()

a = "The Hound of the Baskervilles is the ultimate in detective fiction, pitting the horror of the supernatural against the cold, hard logic of the unflappable Sherlock Holmes. The family is cursed; at least that is the hypothesis when Sir Charles Baskerville is found on the dark and eerie moors near his home, the footprints of some ferocious beast beside his lifeless body. In these wild and haunting surroundings it is down to Sherlock Holmes, and his assistant Dr Watson, to solve the mystery of the killing, before the hound returns to claim the heir to Baskerville Hall."
b = "In the world of cryptocurrency, a node is a computer that connects to a cryptocurrency network. The node supports the cryptocurrency's network through either; relaying transactions, validation or hosting a copy of the blockchain. In terms of relaying transactions each network computer (node) has a copy of the blockchain of the cryptocurrency it supports. When a transaction is made the node creating the transaction broadcasts details of the transaction using encryption to other nodes throughout the node network so that the transaction (and every other transaction) is known."
text = clean(a*30)[:7000]
key = clean("DiegoEmilioBustamanteHenriquez")
ciphertext = e_otp(text, key, frequencies)
key = break_rp(ciphertext, frequencies, abs_distance)
print(key)
"""